In [1]:
import sys, os, time
import h5py
import cv2
import numpy as np
# sys.path.append(os.path.join(os.getcwd(),'../'))
sys.path.append(os.path.join(os.getcwd(),'python/'))
sys.path.append(os.path.join(os.getcwd(),'examples/'))
import darknet as dn
from detectorScipyOpencv import array_to_image, detect2
import matplotlib.pyplot as plt
import matplotlib.patches as pat
import matplotlib.lines as mlines
from math import sqrt


def IOU( x1, y1, w1, h1, x2, y2, w2, h2):
    box1 = ( x1-w1/2.0, y1-h1/2.0, x1+w1/2.0, y1+h1/2.0)
    box2 = ( x2-w2/2.0, y2-h2/2.0, x2+w2/2.0, y2+h2/2.0)
    xA = max( box1[0], box2[0] )
    xB = min( box1[2], box2[2] )
    yA = max( box1[1], box2[1] )
    yB = min( box1[3], box2[3] )

    w = (xB-xA)
    h = (yB-yA)
    if w <= 0 or h <= 0: return 0
    interArea = h * w
    #if interArea <= 0: return 0

    areaA = (box1[2]-box1[0]) * (box1[3]-box1[1])
    areaB = (box2[2]-box2[0]) * (box2[3]-box2[1])
    return interArea / (areaA+areaB-interArea)


/home/liponan/.conda/envs/yolo/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:

rootPath = '/home/liponan/pkg/darknet/'
imagePath = '/home/liponan/data/cs231n/val/'
outputPath = 'debug/'
boxSize = 11
imgSrc = 0 # 0: PNG; 1: CXI; 2: psana
h = 1480
w = 1552

yoloMatch = False
dropletMatch = True
yoloBox = True
dropletBox = True
showText = False

# yolo setting
dataPath = 'cfg/peaks.data'
# cfgPath = 'cfg/newpeaksv2.cfg'
# cfgPath = 'cfg/newpeaks.cfg'
# cfgPath = 'cfg/tiny-peaks-test.cfg'
#weightPath = 'backup/training20171129/peaks_10000.weights' # 51
#weightPath = 'backup/training20171203/peaks_30000.weights' # 11
#weightPath = 'backup/training20171206/peaks.backup'# 21
#weightPath = 'backup/training20171215/peaks.backup'# 7
#weightPath = 'backup/tiny-peaks.backup'
# weightPath = 'backup/newpeaksv2.backup' # latest weights
# weightPath = 'backup/newpeaksv2_900.weights' # latest weights
# weightPath = 'backup/newpeaks_900.weights'
# weightPath = 'backup/newpeaks.backup'
# weightPath = 'backup/peaks_40000.weights'

# cfgPath = 'cfg/newpeaksv2.1.cfg'
# weightPath = 'backup/newpeaksv2.1_final.weights' # latest weights

# cfgPath = 'cfg/newpeaksv4-test.cfg'
# weightPath = 'backup/newpeaksv4_4400.weights' # latest weights

cfgPath = 'cfg/newpeaksv5-test.cfg'
weightPath = 'backup/newpeaksv5_6240.weights' # latest weights

# cfgPath = 'cfg/yolov3-peaks-test.cfg'
# weightPath = 'backup/yolov3-peaks.backup'

# cfgPath = 'cfg/yolov2-peaks-test.cfg'
# weightPath = 'backup/yolov2-peaks.backup'
# weightPath = 'backup/yolov2-peaks_900.weights'

# cfgPath = 'cfg/yolov2-tiny-peaks-test.cfg'
# weightPath = 'backup/yolov2-tiny-peaks.backup'

netReady = False
currentRun = None
imgFlag = False

thresh = 0.20
r = 1

In [ ]:

while (True):
    userInput = raw_input('provide val index: ')
    if userInput == '': break
    idx = str(userInput).split(' ')
    t = int(idx[0])
    if len(idx) > 1:
        thresh = float(idx[1])
    print('detection thresh: {}'.format(str(thresh)))

    if not netReady:
        print('setting up YOLO......')
        #try:
        dn.set_gpu(0)
        net = dn.load_net( cfgPath, weightPath, 0 )
        meta = dn.load_meta( dataPath )
        netReady = True
#         except:
#             print('try again?')
#         continue

    filename = imagePath + '{}.png'.format(str(t).zfill(6))
    if os.path.isfile( filename ):
        time1 = time.time()
        res = dn.detect(net, meta, filename, thresh)
    else:
        print('{} not found, try again?'.format(filename) )
        continue

    time2 = time.time()
    dt = time2 - time1
    print('detected in {} secs'.format(str(dt)))

    txt = open( filename[:-4] + '.txt' ).read().split('\n')
    
    n = len(res)
    m = len(txt)
    
    posX = np.zeros( (m) )
    posY = np.zeros( (m) )
    for u in range(m-1):
        line = txt[u].split(' ')
#         print(line)
        posX[u] = w * float( line[1] ) 
        posY[u] = h * float( line[2] )
    
    print('>> droplet found {} peaks!'.format(str(m)))
    print('>> yolo    found {} peaks!'.format(str(n)))

    if yoloMatch and n*m > 0:
        yoloDist = 0
        for u in range(n):
            minDist = float('Inf')
            minIdx  = -1
            x = r*res[u][2][0]
            y = r*res[u][2][1]
            for v in range(m):
                dist = sqrt( (x - posX[v])**2 + (y - posY[v])**2 )
                if dist < minDist:
                    minDist = dist
                    minIdx = v
            res[u] = ( res[u][0], res[u][1], res[u][2], (minIdx, minDist) )
            yoloDist += minDist
            print('peak {}: {}'.format(str(u), str(res[u][3])))
        yoloDist /= 1.0*n
        print('avg. distance: {0:.2f}'.format(yoloDist))

    if dropletMatch and n*m > 0:
        nn = []
        dropletDist = 0
        recall = 0
        specif = 0
        dropletIOU = 0
        for v in range(m):
            minDist = float('Inf')
            minIdx = -1
            for u in range(n):
                x = r*res[u][2][0]
                y = r*res[u][2][1]
                dist = sqrt( (x - posX[v])**2 + (y - posY[v])**2 )
                if dist < minDist:
                    minDist = dist
                    minIdx = u
            if minIdx == -1: continue
            iou = IOU( posX[v], posY[v], boxSize, boxSize,
                r*res[minIdx][2][0], r*res[minIdx][2][1], r*res[minIdx][2][2], r*res[minIdx][2][3] )
            if iou > 0.0:
                recall += 1
                dropletDist += minDist
                dropletIOU += iou
            nn.append( (minIdx, minDist, iou) )

            print('peak {}: {}'.format(str(v), str(nn[v])))
        if recall > 0:
            dropletIOU /= float(recall)
            dropletDist /= float(recall)
            specif = float(recall) / n
            recall = float(recall) / m

        print('avg. distance: {0:.2f}'.format(dropletDist))
        print('avg. IOU     : {0:.2f}'.format(dropletIOU))
        print('recall       : {0:.2f}'.format(recall))
        print('specificity  : {0:.2f}'.format(specif))


    # plot image
    img = cv2.imread( filename, 0)
    imageShape = img.shape
    h = imageShape[0]
    w = imageShape[1]
    fig, ax = plt.subplots(1)
    cax = ax.imshow(img, vmin=0, vmax=255, cmap="gray")
    cbar = fig.colorbar(cax)

    # plot droplet results
    if dropletBox:
        for i in range(m):
            rect = pat.Rectangle( (posX[i]-(boxSize/2.0), posY[i]-(boxSize/2.0)),
                boxSize, boxSize, color="c", fill=False, linewidth=0.5)
            ax.add_patch(rect)
            if dropletBox and dropletMatch and n*m > 0 and nn[i][2] > 0 and showText:
                ax.text( posX[i], posY[i],
                    str( int(100*nn[i][2]) ), fontsize=4, color='y' )

    # plot yolo results
    if yoloBox:
        for i in range(n):
            if res[i][1] < thresh: continue
            rect = pat.Rectangle( (r*res[i][2][0]-r*res[i][2][2]/2.0, r*res[i][2][1]-r*res[i][2][3]/2.0),
                r*res[i][2][2], r*res[i][2][3], color="m", fill=False, linewidth=0.5,
                alpha=0.2+min(0.8, 0.8/0.40*res[i][1]) )
            ax.add_patch(rect)
            if yoloBox and res[i][1] > 0.01 and showText:
                ax.text( r*res[i][2][0]+r*res[i][2][2]/2.0, r*res[i][2][1]+r*res[i][2][3]/2.0+4,
                    str( int(100*res[i][1]) ), fontsize=4 )

    # plot YOLO vs nearest neighbor
    if yoloMatch and n*m >0:
        for i in range(n):
            if res[i][3][1] > 0.5 * boxSize: continue
            j = res[i][3][0]
            l = mlines.Line2D( [r*res[i][2][0], posX[j]], [r*res[i][2][1], posY[j]],
                color='y', linewidth=0.5 )
            ax.add_line(l)

    # plot DROPLET vs nearest neighbor
    if dropletMatch and n*m > 0:
        for i in range(m):
            if nn[i][1] > 0.5 * boxSize: continue
            j = nn[i][0]
            l = mlines.Line2D( [r*res[j][2][0], posX[i]], [r*res[j][2][1], posY[i]],
                color='y', linewidth=0.5, alpha = 0.5 )
            ax.add_line(l)


    plt.xlim([1, w])
    plt.ylim([h, 1])
        #plt.show()
        #plt.close()
        #plt.savefig(runPath + outputPath +
        #    '_{}.png'.format(str(t).zfill(4)), bbox_inches='tight', dpi=150)
        #plt.clf()
        #plt.cla()
    plt.savefig( rootPath + outputPath + 'val_{}.png'.format(str(t).zfill(6)), bbox_inces='tight', dpi=600)
    #plt.show()
    #plt.clf()
    #plt.cla()
    #plt.close()

    imgFlag = False


provide val index: 777 0.1
detection thresh: 0.1
setting up YOLO......
detected in 1.3918800354 secs
>> droplet found 258 peaks!
>> yolo    found 369 peaks!
peak 0: (257, 0.3583866727995422, 0.2918315946856239)
peak 1: (95, 0.659585830465262, 0.33783179693277027)
peak 2: (326, 0.6838269414981923, 0.27123422911818273)
peak 3: (294, 39.37072356385043, 0)
peak 4: (265, 0.5126467550080611, 0.27045371022008907)
peak 5: (66, 0.41439980250169134, 0.2869542876813938)
peak 6: (288, 0.4227385777448009, 0.11463631600527188)
peak 7: (129, 0.5942015580217715, 0.3073899652072834)
peak 8: (251, 0.5743160953936022, 0.3150429418115583)
peak 9: (279, 11.376152195846899, 0)
peak 10: (234, 0.9114419439806155, 0.2879849999662838)
peak 11: (311, 0.7241784252806689, 0.28072794151619535)
peak 12: (279, 0.9290467402516989, 0.2774335476984704)
peak 13: (234, 12.64432579097292, 0)
peak 14: (347, 0.4737455374000654, 0.264300699712027)
peak 15: (347, 12.614218498208068, 0)
peak 16: (103, 0.838767898604377, 0.28406

peak 203: (75, 0.3376803322553277, 0.3065447499436254)
peak 204: (195, 0.8699545054487084, 0.3064318120913345)
peak 205: (215, 0.8408100360789685, 0.2968165102117837)
peak 206: (45, 0.9510971690952909, 0.3446670963624058)
peak 207: (238, 0.7998390156909375, 0.28593143565670226)
peak 208: (16, 0.5223312186255954, 0.3574791073385223)
peak 209: (74, 0.957956924244982, 0.31367694712129424)
peak 210: (183, 0.42003016323149167, 0.3146799575317255)
peak 211: (9, 0.1448115260297664, 0.33321726107664484)
peak 212: (8, 0.481284695621768, 0.3427670889628974)
peak 213: (65, 0.21265017695675348, 0.3380977960055059)
peak 214: (259, 0.8154358115865684, 0.4339438824735402)
peak 215: (101, 0.5560069966495126, 0.3198736078956214)
peak 216: (134, 0.54804360459639, 0.29222623471532116)
peak 217: (36, 0.29359404521905597, 0.32793855142447437)
peak 218: (194, 0.7183490501696917, 0.2840645949975169)
peak 219: (367, 1.299015773038804, 0.11048002015155918)
peak 220: (0, 25.214928749316655, 0)
peak 221: (0, 0.2

peak 113: (28, 0.7713369808154226, 0.31849309072006216)
peak 114: (106, 0.8640033050846492, 0.2825406552715998)
peak 115: (88, 0.7982782859284286, 0.32302553429267694)
peak 116: (286, 0.12623194749577785, 0.2789852002772947)
peak 117: (307, 0.6842580625981447, 0.2775520394839904)
peak 118: (306, 0.6078732161424648, 0.2868274231406311)
peak 119: (54, 0.2767185582446226, 0.32339965711849983)
peak 120: (449, 0.6032417140270396, 0.08769743451959626)
peak 121: (200, 1.2637939453125, 0.3238660336256461)
peak 122: (319, 1.1541871746495742, 0.31389053942370054)
peak 123: (359, 1.6505094271003207, 0.3384511185993928)
peak 124: (124, 1.0000974730580963, 0.3146265206160926)
peak 125: (285, 0.5554964888899082, 0.31586584806974155)
peak 126: (214, 0.6723365307419675, 0.3421844990032861)
peak 127: (237, 1.0438943267019147, 0.30799550869539216)
peak 128: (493, 53.05407955240409, 0)
peak 129: (186, 0.5350478027315483, 0.3041455406433061)
peak 130: (176, 1.1505145956018676, 0.29486516564227616)
peak 13

detected in 1.12827515602 secs
>> droplet found 258 peaks!
>> yolo    found 44 peaks!
peak 0: (17, 188.92135474517985, 0)
peak 1: (17, 111.79405578181974, 0)
peak 2: (17, 110.50097581778986, 0)
peak 3: (27, 119.70908645877381, 0)
peak 4: (17, 102.4044265860522, 0)
peak 5: (17, 95.77644045043465, 0)
peak 6: (27, 81.67887037750795, 0)
peak 7: (12, 107.04864724171661, 0)
peak 8: (17, 55.853378348962686, 0)
peak 9: (17, 54.272449721508075, 0)
peak 10: (27, 42.1652668548263, 0)
peak 11: (27, 67.93653555859571, 0)
peak 12: (17, 44.69406660608597, 0)
peak 13: (27, 30.987363145958316, 0)
peak 14: (27, 39.551693082800604, 0)
peak 15: (27, 48.75560865105478, 0)
peak 16: (27, 30.907217659479368, 0)
peak 17: (43, 61.0878777950441, 0)
peak 18: (17, 0.9131044360444225, 0.3334348017007403)
peak 19: (17, 12.281370067168769, 0)
peak 20: (17, 24.318445658838996, 0)
peak 21: (17, 34.30408284801031, 0)
peak 22: (27, 25.051106209069665, 0)
peak 23: (27, 0.071602289994264, 0.32843158848254084)
peak 24: (27,

provide val index: 777 0.3
detection thresh: 0.3
detected in 1.14445281029 secs
>> droplet found 258 peaks!
>> yolo    found 142 peaks!
peak 0: (66, 93.40599469451676, 0)
peak 1: (95, 0.659585830465262, 0.33783179693277027)
peak 2: (95, 12.236679410648975, 0)
peak 3: (123, 82.26956132201651, 0)
peak 4: (66, 12.414044255376185, 0)
peak 5: (66, 0.41439980250169134, 0.2869542876813938)
peak 6: (123, 0.4991031016721427, 0.2801466052048777)
peak 7: (129, 0.5942015580217715, 0.3073899652072834)
peak 8: (66, 40.85212694017482, 0)
peak 9: (103, 27.888920927385836, 0)
peak 10: (103, 30.28092766365843, 0)
peak 11: (27, 67.93653555859571, 0)
peak 12: (103, 18.01070452226982, 0)
peak 13: (103, 20.424843566419998, 0)
peak 14: (27, 39.551693082800604, 0)
peak 15: (27, 48.75560865105478, 0)
peak 16: (103, 0.838767898604377, 0.28406478714823646)
peak 17: (127, 0.2694108307878077, 0.30306836615495214)
peak 18: (17, 0.9131044360444225, 0.3334348017007403)
peak 19: (17, 12.281370067168769, 0)
peak 20: (9

...

peak 146: (154, 0.34529797471683193, 0.29053136461001006)
peak 147: (156, 0.14867105743876993, 0.2985263224773082)
peak 148: (71, 0.7742584813044642, 0.2946307349359189)
peak 149: (406, 0.17602828006242993, 0.1375630565359778)
peak 150: (30, 0.8250475350506304, 0.3175251299681759)
peak 151: (3, 0.7792738324597214, 0.3378374266660332)
peak 152: (322, 0.5917662797153245, 0.12908973402001123)
peak 153: (336, 0.740958178405664, 0.12405524907241636)
peak 154: (226, 0.37546763499759545, 0.27706628756399265)
peak 155: (19, 1.1254596433491586, 0.31576515305642183)
peak 156: (153, 0.05836097454973902, 0.30046859272311655)
peak 157: (1, 0.6062235442084124, 0.31756027167771694)
peak 158: (83, 1.025186801929657, 0.34309599809107155)
peak 159: (77, 0.7814171069956005, 0.28813426553648985)
peak 160: (110, 0.3497537336187696, 0.30675765371647273)
peak 161: (464, 0.5469056027228595, 0.25173569121942224)
peak 162: (235, 0.2050232700372623, 0.12000882389970721)
peak 163: (68, 0.40160094482542086, 0.2845

peak 106: (7, 0.6208893159086512, 0.31315603049100454)
peak 107: (298, 0.35328994444389916, 0.30411538734761956)
peak 108: (256, 0.6123662093312375, 0.2732223208174542)
peak 109: (152, 8.083142239537136, 0.03866555036972463)
peak 110: (152, 0.3663134380580611, 0.27169399690323687)
peak 111: (59, 0.8950637566086702, 0.2948175668977967)
peak 112: (33, 0.560099194525857, 0.3171802849602793)
peak 113: (28, 0.7713369808154226, 0.31849309072006216)
peak 114: (106, 0.8640033050846492, 0.2825406552715998)
peak 115: (88, 0.7982782859284286, 0.32302553429267694)
peak 116: (286, 0.12623194749577785, 0.2789852002772947)
peak 117: (519, 0.5999063546954672, 0.14445032902262975)
peak 118: (306, 0.6078732161424648, 0.2868274231406311)
peak 119: (54, 0.2767185582446226, 0.32339965711849983)
peak 120: (449, 0.6032417140270396, 0.08769743451959626)
peak 121: (200, 1.2637939453125, 0.3238660336256461)
peak 122: (319, 1.1541871746495742, 0.31389053942370054)
peak 123: (695, 0.6684362242464907, 0.1314383786

provide val index: 777 0.01
detection thresh: 0.01
detected in 1.19280290604 secs
>> droplet found 258 peaks!
>> yolo    found 1101 peaks!
peak 0: (257, 0.3583866727995422, 0.2918315946856239)
peak 1: (95, 0.659585830465262, 0.33783179693277027)
peak 2: (387, 0.6091145518992512, 0.11523366338304952)
peak 3: (572, 0.493104425922445, 0.12051249573788625)
peak 4: (265, 0.5126467550080611, 0.27045371022008907)
peak 5: (66, 0.41439980250169134, 0.2869542876813938)
peak 6: (288, 0.4227385777448009, 0.11463631600527188)
peak 7: (402, 0.45380877471169206, 0.10872308412708362)
peak 8: (251, 0.5743160953936022, 0.3150429418115583)
peak 9: (597, 0.7977166178451912, 0.12696550055119407)
peak 10: (234, 0.9114419439806155, 0.2879849999662838)
peak 11: (509, 0.4256072577470894, 0.11897961694617228)
peak 12: (562, 0.6845740948761674, 0.13611904049122875)
peak 13: (581, 0.5346653099951698, 0.1390636776393534)
peak 14: (347, 0.4737455374000654, 0.264300699712027)
peak 15: (408, 1.1706671947767147, 0.273

peak 145: (919, 1.1257509708014939, 0.16916291408455392)
peak 146: (154, 0.34529797471683193, 0.29053136461001006)
peak 147: (156, 0.14867105743876993, 0.2985263224773082)
peak 148: (71, 0.7742584813044642, 0.2946307349359189)
peak 149: (406, 0.17602828006242993, 0.1375630565359778)
peak 150: (30, 0.8250475350506304, 0.3175251299681759)
peak 151: (693, 0.6360267964921793, 0.14546756221976798)
peak 152: (322, 0.5917662797153245, 0.12908973402001123)
peak 153: (336, 0.740958178405664, 0.12405524907241636)
peak 154: (226, 0.37546763499759545, 0.27706628756399265)
peak 155: (19, 1.1254596433491586, 0.31576515305642183)
peak 156: (153, 0.05836097454973902, 0.30046859272311655)
peak 157: (1089, 0.5427703256867888, 0.07749658763858905)
peak 158: (544, 0.7461938441375323, 0.15056959224515715)
peak 159: (77, 0.7814171069956005, 0.28813426553648985)
peak 160: (1097, 0.28467697371411027, 0.057644177990885094)
peak 161: (464, 0.5469056027228595, 0.25173569121942224)
peak 162: (235, 0.2050232700372

peak 220: (0, 25.214928749316655, 0)
peak 221: (0, 0.2112959849875113, 0.32418755629904517)
peak 222: (109, 0.534560949902468, 0.32043894835345227)
peak 223: (297, 0.8204306518808473, 0.2772780419039738)
peak 224: (44, 0.49455414331283976, 0.31108811128934044)
peak 225: (60, 0.5197894722485535, 0.3083388805530451)
peak 226: (289, 0.6460010489809342, 0.11731707041920951)
peak 227: (175, 1.2926607312494005, 0.2883159832826366)
peak 228: (168, 1.1120470208435205, 0.2702922255042636)
peak 229: (149, 0.7514135135286975, 0.289880989335066)
peak 230: (32, 0.5264546053259777, 0.30441284823170034)
peak 231: (85, 27.658511590592, 0)
peak 232: (80, 0.24114263393732205, 0.33412473568318063)
peak 233: (31, 0.9714913037048811, 0.3430831927407931)
peak 234: (5, 0.6911403769819361, 0.3174211970584182)
peak 235: (99, 0.06504722091551672, 0.3148286203108872)
peak 236: (80, 15.04623555768692, 0)
peak 237: (22, 0.18338119092617708, 0.3252630762806968)
peak 238: (91, 0.9404158150510322, 0.2917948839838738)

##### 

In [ ]:
1100



###### 